In [1]:
!pip install psycopg2

In [2]:
import re
import psycopg2
import numpy as np
import itertools
from itertools import combinations_with_replacement as cmb

In [3]:
#we need to mannually write input text file name
f = open("input1.txt", "r")
data = f.read()

myarray = []
myarr = re.split('[-:]',data)
for i in myarr:
    j = i.replace(' ','')
    myarray.append(j)
S1 = [myarray[1].strip('\nNUMBER OF GROUPING VARIABLES(n)')]
n1 = [myarray[2].strip('\nGROUPING ATTRIBUTES(V)')]
V1 = [myarray[3].strip('\n3\nGROUPING ATTRIBUTES(V)\nF')]
F1 = [myarray[5].strip('\nSELECT CONDITION')]
C1 = [myarray[7].strip('\nHAVING_CONDITION(G)')]
G = [myarray[8].strip('\nHAVING_CONDITION(G)')]
#if no having clause we need to initialize G as '' and comment the G = [myarray[8].strip('\nHAVING_CONDITION(G)')]
# G = ''
print(myarray)


['SELECTATTRIBUTE(S)', '\ncust,prod,1_sum_quant,1_count_quant,1_avg_quant,2_sum_quant,3_sum_quant\nNUMBEROFGROUPINGVARIABLES(n)', '\n1,2,3\nGROUPINGATTRIBUTES(V)', '\ncust,prod\nF', 'VECT([F])', '\n1_sum_quant,1_avg_quant,2_avg_quant,3_sum_quant,3_avg_quant,3_sum_quant\nSELECTCONDITION', 'VECT([Ïƒ])', "\n1.cust=custand1.state='NY'\n2.cust=custand2.state='CT'\n3.cust=custand3.state='NJ'\nHAVING_CONDITION(G)", '\n1_avg_quant>2_avg_quantand1_avg_quant>=3_avg_quant']


In [4]:
S1

['cust,prod,1_sum_quant,1_count_quant,1_avg_quant,2_sum_quant,3_sum_quant']

In [5]:
S=[]
for i in S1:
 S +=i.split(",")
print(S)

['cust', 'prod', '1_sum_quant', '1_count_quant', '1_avg_quant', '2_sum_quant', '3_sum_quant']


In [6]:
n=[]
for i in n1:
     n +=i.split(",")
print(n)  
    

['1', '2', '3']


In [7]:
V = []
for i in V1:
    V +=i.split(",")
print(V)

['cust', 'prod']


In [8]:
F=[]
for i in F1:
 F +=i.split(",")
print(F)

['1_sum_quant', '1_avg_quant', '2_avg_quant', '3_sum_quant', '3_avg_quant', '3_sum_quant']


In [9]:
C = re.split('[-\n]',C1[0])
(C)


["1.cust=custand1.state='NY'",
 "2.cust=custand2.state='CT'",
 "3.cust=custand3.state='NJ'"]

In [10]:
table = "sales"
dbname = "DBMS2"
host = "localhost"
user = "postgres"
password = "yash1712"
port = "5432"
try:
    connection = psycopg2.connect(user=user, password=password, host=host, port=port, database=dbname)
    query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '" + table + "';"
    pointer = connection.cursor()
    pointer.execute(query)
    if pointer.rowcount == 0:
            print("Database is Connected yipee %s " % table)
            sys.exit(0)
    schema = pointer.fetchall()
except(Exception, psycopg2.Error) as error:
            print("Connection failed !", error)
            sys.exit(0)
            
print("'%s' table" % format(table))

print("ATTRIBUTES".ljust(14, " "), "DTYPE".ljust(15, " "))
for i in schema:
    print("-", i[0].ljust(12, " "), i[1].ljust(15, " "))


'sales' table
ATTRIBUTES     DTYPE          
- month        integer        
- day          integer        
- year         integer        
- quant        integer        
- prod         character varying
- state        character      
- cust         character varying


In [11]:
PG_log_info = {'user':user, 'password':password,'host':host, 'port':port, 'database':dbname}
initial_list=['user', 'password', 'host', 'port', 'database']

In [12]:
schema=[('cust', 'character varying'),('prod', 'character varying'),('day', 'integer'),('month', 'integer'),('year', 'integer'),('state', 'character'),('quant', 'integer')]

In [13]:
schema

[('cust', 'character varying'),
 ('prod', 'character varying'),
 ('day', 'integer'),
 ('month', 'integer'),
 ('year', 'integer'),
 ('state', 'character'),
 ('quant', 'integer')]

In [14]:
Tcolumns = []
for x in schema:
    Tcolumns.append(x[0])
print(Tcolumns)

['cust', 'prod', 'day', 'month', 'year', 'state', 'quant']


In [15]:
Vcolumns=[]
for i in V:
     Vcolumns +=i.split(",")
        

print(Vcolumns)

['cust', 'prod']


In [16]:
Mfcolumns = []
for i in Vcolumns:
    Mfcolumns.append(i)
    if "avg" in i:
        Mfcolumns.append(i.replace('avg','sum'))
        Mfcolumns.append(i.replace('avg','count'))
    
for x in F:
    Mfcolumns.append(x)
    if "avg" in x:
        Mfcolumns.append(x.replace('avg','sum'))
        Mfcolumns.append(x.replace('avg','count'))
Mfcolumns


['cust',
 'prod',
 '1_sum_quant',
 '1_avg_quant',
 '1_sum_quant',
 '1_count_quant',
 '2_avg_quant',
 '2_sum_quant',
 '2_count_quant',
 '3_sum_quant',
 '3_avg_quant',
 '3_sum_quant',
 '3_count_quant',
 '3_sum_quant']

In [17]:
Mfdata = {}
Mfdata1={}
for i in Mfcolumns:
    if('count' in i):
       
        Mfdata[i] = 0
    elif('quant' in i):
        Mfdata[i] = 0.0
    else:
        Mfdata[i] = ''
        

Mfdata
        

        

{'cust': '',
 'prod': '',
 '1_sum_quant': 0.0,
 '1_avg_quant': 0.0,
 '1_count_quant': 0,
 '2_avg_quant': 0.0,
 '2_sum_quant': 0.0,
 '2_count_quant': 0,
 '3_sum_quant': 0.0,
 '3_avg_quant': 0.0,
 '3_count_quant': 0}

In [18]:


Ccolumns = [row.strip().split("and") for row in C if len(row) > 2]
Ccolumns

[['1.cust=cust', "1.state='NY'"],
 ['2.cust=cust', "2.state='CT'"],
 ['3.cust=cust', "3.state='NJ'"]]

In [19]:
# selection format
prinitngFormat = {"''": "{:<12}", 0: "{:>17}", 0.0: "{:>17}"}
Tableformatting = "print('='*100)\n"
format_ = "print('{:<8}|"
cols = "'.format('index'"
# getting MFdata
for col in S:
    try: 
        format_ += prinitngFormat[MF_dtypes[i]] + '|'
        cols += ",'" + col + "'"
    except:
        format_ += prinitngFormat[0] + '|'
        cols += ",'" + col + "'"
Tableformatting += format_ + cols + '))\n' + "print('='*100)\n"

In [20]:
import_text = 'import psycopg2\nimport sys\nfrom collections import defaultdict'
mylist = list(PG_log_info.keys())
connection = '\ntry:\n    connection = psycopg2.connect('
for i in mylist:
    connection += str(i) + "='" + str(PG_log_info[i]) + "', "
connection += ''')\n    query = "SELECT * FROM sales"\n    pointer = connection.cursor()\n    pointer.execute(query)\n    if (pointer.rowcount) == 0:\n        print("Connection failed !")\n        sys.exit(0)
                    \n    dataItems = pointer.fetchall()\nexcept(Exception, psycopg2.Error) as error:\n    print("Connection failed to database ==>", error)\n    sys.exit(0)\n
                    '''

print(connection)


try:
    connection = psycopg2.connect(user='postgres', password='yash1712', host='localhost', port='5432', database='DBMS2', )
    query = "SELECT * FROM sales"
    pointer = connection.cursor()
    pointer.execute(query)
    if (pointer.rowcount) == 0:
        print("Connection failed !")
        sys.exit(0)
                    
    dataItems = pointer.fetchall()
except(Exception, psycopg2.Error) as error:
    print("Connection failed to database ==>", error)
    sys.exit(0)

                    


In [21]:
MFVector = "MFVector = defaultdict(lambda: "
MFVector +=  str(Mfdata) + ', '
MFVector = MFVector[:-2] + ')\n' 
MFVector += 'MF_idx=0\nMFMap = {}\n'
print(MFVector)




MFVector = defaultdict(lambda: {'cust': '', 'prod': '', '1_sum_quant': 0.0, '1_avg_quant': 0.0, '1_count_quant': 0, '2_avg_quant': 0.0, '2_sum_quant': 0.0, '2_count_quant': 0, '3_sum_quant': 0.0, '3_avg_quant': 0.0, '3_count_quant': 0})
MF_idx=0
MFMap = {}



In [22]:

d = "for i in range(len(dataItems)):\n\ttemp = {x[0]:x[1] for x in zip(" + str(Tcolumns) + ",dataItems[i])}\n" + "\n\tchecker ="+"+".join(['str(temp' + str([i]) + ")" for i in Vcolumns])+ "\n"
Mfmapinitial = []
Mfmapinitial.append("\tif (" + "MFMap.get(checker)" + ")== None:\n")
Mfmapinitial.append( "\t\t" + "\n\t\t".join(['MFVector[MF_idx]' + str([i]) + '=temp' + str([i]) for i in Vcolumns]) + '\n')

for k in cmb(Vcolumns, len(Vcolumns)):
    t = '\t\tMFMap['
    if len(set(k)) > 1:
            t = '\t\tMFMap[' 
            t += " + ".join(['str(temp' + str([j]) + ")" for j in list(k)])   
            t += ']=MF_idx\n'
            t += '\t\tMFMap['  
            t += " + ".join(['str(temp' + str([j]) + ")" for j in list(k)[::-1]])   
            t += ']=MF_idx\n'
    else:  
        t += 'str(temp' + str(list(set(k))) + ")"  
        t += ']=MF_idx\n'
    Mfmapinitial.append(t)

MfMap = d + "".join(Mfmapinitial) + "\t\tMF_idx+=1 # new idx\n\n"  

print(MfMap)
            

for i in range(len(dataItems)):
	temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}

	checker =str(temp['cust'])+str(temp['prod'])
	if (MFMap.get(checker))== None:
		MFVector[MF_idx]['cust']=temp['cust']
		MFVector[MF_idx]['prod']=temp['prod']
		MFMap[str(temp['cust'])]=MF_idx
		MFMap[str(temp['cust']) + str(temp['prod'])]=MF_idx
		MFMap[str(temp['prod']) + str(temp['cust'])]=MF_idx
		MFMap[str(temp['prod'])]=MF_idx
		MF_idx+=1 # new idx




In [23]:
list(k)

['prod', 'prod']

In [24]:
logic_map = {">=": " >= ", "<=": " <= ", "=": " == ", "<>": ' != ', ">": " > ", "<": " < "}
MF_update = "for groupingvariable in " + str(n) + ":\n"
d2 = "\t\tfor i in range(len(dataItems)):\n\t\t\ttemp = {x[0]:x[1] for x in zip(" + str(Tcolumns) + ",dataItems[i])}\n"
for first in range(len(n)):
    Map_index = '\t\t\tindex = MFMap['
    indexing_cols = []
    Condition = '\t\t\tif ('
    conditioning_cols = []
    Update = ''
    updating_cols = []
    res = []
    for piece in Ccolumns[first]:
        if ('=' in piece) and (">=" not in piece) and ("<=" not in piece):  
            t = piece.split("=")[-1].strip()
          
            if t in Vcolumns:  
                indexing_cols.append("str(temp['" + t + "'])") 
                t = ''
        for key in logic_map.keys(): 
            
            if key in piece:  
                x = [i.strip() for i in piece.replace(key, logic_map[key]).split(" ") if len(i.strip()) >= 1] 
                
                
                break  # only replace once

        if x[2] in Vcolumns: 
                
            conditioning_cols.append("(temp['" + x[0][(x[0].find('.') + 1):] + "']" + x[1] + "MFVector[index][" + "'" + x[2] + "'])")  
        else:
                
                
            conditioning_cols.append("(temp['" + x[0][(x[0].find('.') + 1):] + "']" + x[1] + x[2] + ")")  
                
    
    agg_cols = [col for col in Mfcolumns if (n[first]) == col[0]] 
    
    gv_attrs_set = list(set([gv_attrs.split('_')[2] for gv_attrs in agg_cols]))  
    for attrs in gv_attrs_set:  
        agg_attr_cols = [each for each in agg_cols if each.find(attrs) > 1]
        updating_col = []  
        for agg_col in agg_attr_cols:
            
            y = agg_col.split('_')  
            
            if 'avg' == y[-2]:
                updating_col = [] 
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'count') + "']+=1\n")
                updating_col.append( "\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'sum') + "']+=" + "temp['" + y[-1] + "']\n")
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col + "']=" + "MFVector[index]['" + agg_col.replace('avg','sum') + "']"+ '/' + "MFVector[index]['" + agg_col.replace('avg', 'count') + "']\n")
                
                break
                
            elif ('sum' == y[-2]):
                
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'sum') + "']+=" + "temp['" + y[-1] + "']\n")
            elif ('count' == y[-2]):
                updating_col.append("\t\t\t\tMFVector[index]['" + agg_col.replace('avg', 'count') + "']+=1\n")
            else:
                print(":< Some error occured", y[-2])
        updating_cols.extend(updating_col)
    gv_judge = "\tif (groupingvariable) == " + str("'" + n[first] + "'") + ":\n"
    Map_index += "+".join(indexing_cols) + "]\n"  
    Condition += " and ".join(conditioning_cols) + "):\n"  
    Update += "".join(updating_cols) + "\n"
    first_d = gv_judge + d2 + Map_index + Condition + Update

    MF_update += first_d
print(MF_update)

for groupingvariable in ['1', '2', '3']:
	if (groupingvariable) == '1':
		for i in range(len(dataItems)):
			temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}
			index = MFMap[str(temp['cust'])]
			if ((temp['cust']==MFVector[index]['cust']) and (temp['state']=='NY')):
				MFVector[index]['1_count_quant']+=1
				MFVector[index]['1_sum_quant']+=temp['quant']
				MFVector[index]['1_avg_quant']=MFVector[index]['1_sum_quant']/MFVector[index]['1_count_quant']

	if (groupingvariable) == '2':
		for i in range(len(dataItems)):
			temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}
			index = MFMap[str(temp['cust'])]
			if ((temp['cust']==MFVector[index]['cust']) and (temp['state']=='CT')):
				MFVector[index]['2_count_quant']+=1
				MFVector[index]['2_sum_quant']+=temp['quant']
				MFVector[index]['2_avg_quant']=MFVector[index]['2_sum_quant']/MFVector[index]['2_count_quant']

	if (groupi

In [25]:
def havingfunction(x):
    out = []
    for frac in x:  
        if ("_" in frac) and len(re.findall('(\w+)[\+\-\/\*]', frac)) >= 1:
            col = re.findall('(\w+)[\+\-\/\*]', frac)[0] 
            compute_symbol = re.findall('[\+\-\/\*]', frac)
            suffix = frac[len(col) + 1::]  
            if re.findall('[\D]+', suffix):  
                out.extend(["MFVector[index]['" + col + "']" + compute_symbol[0] + "MFVector[index]['" + suffix + "']"])
            else:  
                out.extend(["MFVector[index]['" + col + "']" + compute_symbol[0] + suffix])
        elif ("_" in frac):  
            out.extend(["MFVector[index]['" + frac + "']"])
        else:  
            out.append(frac)
    return "".join(out)

In [26]:
def having_function(H_F):
    having = []
    for x in H_F[0].split('and'):
        piece = x[:]
        for key in logic_map.keys():  
            if key in piece:  
                j = [i.strip() for i in piece.replace(key, logic_map[key]).split(" ") if len(i.strip()) >= 1]
               
                break  
        
        out = havingfunction(j)
        having.append("".join(out))  
    return " and ".join(having)


In [27]:
if(G != ''):
    havingoutput = 'output=0\n'
    havingoutput += "for index in range(len(MFVector)):\n"
    havingoutput += "\t if " + having_function(G) + ":\n"
    havingoutput += "\t\t\toutput += 1\n"
    havingoutput += "\t\t\tprint(end='{:<6}|'.format(output))\n"
    print(havingoutput)

output=0
for index in range(len(MFVector)):
	 if MFVector[index]['1_avg_quant']>MFVector[index]['2_avg_quant'] and MFVector[index]['1_avg_quant']>=MFVector[index]['3_avg_quant']:
			output += 1
			print(end='{:<6}|'.format(output))



In [28]:
        
Selection_text = ''
for col in S:
    try:  
        
        if type(Mfdata[col]) == str:
            Selection_text += "\t\t\tprint('{:<17}|'.format(MFVector[index]['" + col + "']), end='')\n"
        elif (type(Mfdata[col]) == float):
            Selection_text += "\t\t\tprint('{:>17.2f}|'.format(MFVector[index]['" + col + "']), end='')\n"
        elif (type(Mfdata[col]) == int):
            Selection_text += "\t\t\tprint('{:>17.0f}|'.format(MFVector[index]['" + col + "']), end='')\n"
        
    except:  # the combined output
        Selection_text += "\t\t\tprint('{:>17.2f}|'.format(" + havingfunction([col]) + "), end='')\n"  # it can only process list

Selection_text += "\t\t\tprint('')"

print(Selection_text)

			print('{:<17}|'.format(MFVector[index]['cust']), end='')
			print('{:<17}|'.format(MFVector[index]['prod']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['1_sum_quant']), end='')
			print('{:>17.0f}|'.format(MFVector[index]['1_count_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['1_avg_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['2_sum_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['3_sum_quant']), end='')
			print('')


In [29]:
myresult = '# Import and Connecting DB\n'
myresult += import_text + '\n'
myresult += connection + '\n'
myresult += '# Initializing MF_Vector\n'
myresult += MFVector  + '\n'
myresult += '# Initializing Map for MF_Structure\n'
myresult += MfMap + '\n'
myresult += '# Updating for MF_Structure\n'
myresult += MF_update + '\n'
myresult += '# Outputing\n'
myresult += Tableformatting
myresult += '# Having\n'
if(G != ''):
    myresult += havingoutput + '\n'
myresult += Selection_text + '\n'

In [30]:
#the output file will be updated as the input changes. refresh the output.py file
with open('output.py', 'w') as f:
    f.write(myresult)
v = open("output.py", "r")
finaldata = v.read()
print(finaldata)

# Import and Connecting DB
import psycopg2
import sys
from collections import defaultdict

try:
    connection = psycopg2.connect(user='postgres', password='yash1712', host='localhost', port='5432', database='DBMS2', )
    query = "SELECT * FROM sales"
    pointer = connection.cursor()
    pointer.execute(query)
    if (pointer.rowcount) == 0:
        print("Connection failed !")
        sys.exit(0)
                    
    dataItems = pointer.fetchall()
except(Exception, psycopg2.Error) as error:
    print("Connection failed to database ==>", error)
    sys.exit(0)

                    
# Initializing MF_Vector
MFVector = defaultdict(lambda: {'cust': '', 'prod': '', '1_sum_quant': 0.0, '1_avg_quant': 0.0, '1_count_quant': 0, '2_avg_quant': 0.0, '2_sum_quant': 0.0, '2_count_quant': 0, '3_sum_quant': 0.0, '3_avg_quant': 0.0, '3_count_quant': 0})
MF_idx=0
MFMap = {}

# Initializing Map for MF_Structure
for i in range(len(dataItems)):
	temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day'

# copying the whole output file from "output.py" and pasting it here 
#as our output file is .py and jupyter notebook support extenstion .ipynb we are running out output file here 
#due ti security reason we cannot save our out with .ipynb extension for code 
#thus,need to run in jupyter notebook we need to copy and paste thr output.py file

In [33]:
# Import and Connecting DB
import psycopg2
import sys
from collections import defaultdict

try:
    connection = psycopg2.connect(user='postgres', password='yash1712', host='localhost', port='5432', database='DBMS2', )
    query = "SELECT * FROM sales"
    pointer = connection.cursor()
    pointer.execute(query)
    if (pointer.rowcount) == 0:
        print("Connection failed !")
        sys.exit(0)
                    
    dataItems = pointer.fetchall()
except(Exception, psycopg2.Error) as error:
    print("Connection failed to database ==>", error)
    sys.exit(0)

                    
# Initializing MF_Vector
MFVector = defaultdict(lambda: {'cust': '', 'prod': '', '1_sum_quant': 0.0, '1_avg_quant': 0.0, '1_count_quant': 0, '2_avg_quant': 0.0, '2_sum_quant': 0.0, '2_count_quant': 0, '3_sum_quant': 0.0, '3_avg_quant': 0.0, '3_count_quant': 0})
MF_idx=0
MFMap = {}

# Initializing Map for MF_Structure
for i in range(len(dataItems)):
	temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}

	checker =str(temp['cust'])+str(temp['prod'])
	if (MFMap.get(checker))== None:
		MFVector[MF_idx]['cust']=temp['cust']
		MFVector[MF_idx]['prod']=temp['prod']
		MFMap[str(temp['cust'])]=MF_idx
		MFMap[str(temp['cust']) + str(temp['prod'])]=MF_idx
		MFMap[str(temp['prod']) + str(temp['cust'])]=MF_idx
		MFMap[str(temp['prod'])]=MF_idx
		MF_idx+=1 # new idx


# Updating for MF_Structure
for groupingvariable in ['1', '2', '3']:
	if (groupingvariable) == '1':
		for i in range(len(dataItems)):
			temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}
			index = MFMap[str(temp['cust'])]
			if ((temp['cust']==MFVector[index]['cust']) and (temp['state']=='NY')):
				MFVector[index]['1_count_quant']+=1
				MFVector[index]['1_sum_quant']+=temp['quant']
				MFVector[index]['1_avg_quant']=MFVector[index]['1_sum_quant']/MFVector[index]['1_count_quant']

	if (groupingvariable) == '2':
		for i in range(len(dataItems)):
			temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}
			index = MFMap[str(temp['cust'])]
			if ((temp['cust']==MFVector[index]['cust']) and (temp['state']=='CT')):
				MFVector[index]['2_count_quant']+=1
				MFVector[index]['2_sum_quant']+=temp['quant']
				MFVector[index]['2_avg_quant']=MFVector[index]['2_sum_quant']/MFVector[index]['2_count_quant']

	if (groupingvariable) == '3':
		for i in range(len(dataItems)):
			temp = {x[0]:x[1] for x in zip(['cust', 'prod', 'day', 'month', 'year', 'state', 'quant'],dataItems[i])}
			index = MFMap[str(temp['cust'])]
			if ((temp['cust']==MFVector[index]['cust']) and (temp['state']=='NJ')):
				MFVector[index]['3_count_quant']+=1
				MFVector[index]['3_sum_quant']+=temp['quant']
				MFVector[index]['3_avg_quant']=MFVector[index]['3_sum_quant']/MFVector[index]['3_count_quant']


# Outputing
print('='*100)
print('{:<8}|{:>17}|{:>17}|{:>17}|{:>17}|{:>17}|{:>17}|{:>17}|'.format('index','cust','prod','1_sum_quant','1_count_quant','1_avg_quant','2_sum_quant','3_sum_quant'))
print('='*100)
# Having
output=0
for index in range(len(MFVector)):
	 if MFVector[index]['1_avg_quant']>MFVector[index]['2_avg_quant'] and MFVector[index]['1_avg_quant']>=MFVector[index]['3_avg_quant']:
			output += 1
			print(end='{:<6}|'.format(output))

			print('{:<17}|'.format(MFVector[index]['cust']), end='')
			print('{:<17}|'.format(MFVector[index]['prod']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['1_sum_quant']), end='')
			print('{:>17.0f}|'.format(MFVector[index]['1_count_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['1_avg_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['2_sum_quant']), end='')
			print('{:>17.2f}|'.format(MFVector[index]['3_sum_quant']), end='')
			print('')


index   |             cust|             prod|      1_sum_quant|    1_count_quant|      1_avg_quant|      2_sum_quant|      3_sum_quant|
1     |Knuth            |Butter           |         89584.00|               36|          2488.44|        114824.00|         85714.00|
2     |Bloom            |Bread            |         84480.00|               32|          2640.00|        123326.00|        137394.00|
